## Motivation

explain why we are doing this project and why it's a topic worth doing data science on

# Part 1: Getting/Formatting the Data

For this project, we decided to use the [House Office Expenditure Data](https://www.propublica.org/datastore/dataset/house-office-expenditures) from ProPublica as it contains well formatted data about house expenditures from 2009 to 2018. The main downside of this dataset is that it is missing data from the most recent midterm election, but we still have almost 10 years of data to work with.

In [24]:
import pandas as pd
# just finna test with one of the files to see what happens
frames = []
#manually add stuff for 2009 since only Q3 and Q4 are present
frames.append(pd.read_csv('house-expenditure/2009Q3-house-disburse-summary.csv').dropna(subset = ['BIOGUIDE_ID']))
frames.append(pd.read_csv('house-expenditure/2009Q4-house-disburse-summary.csv').dropna(subset = ['BIOGUIDE_ID']))


#automate the dataframes from 2010 to 2017
for i in range(2010, 2018):
    for j in range(1, 5):
        df = pd.read_csv('house-expenditure/' + str(i) + 'Q' + str(j) +'-house-disburse-summary.csv')
        frames.append(df.dropna(subset = ['BIOGUIDE_ID']))

# manually add stuff for 2018 since only Q1 is present
frames.append(pd.read_csv('house-expenditure/2018Q1-house-disburse-summary.csv').dropna(subset = ['BIOGUIDE_ID']))

house_data = pd.concat(frames)
house_data

,BIOGUIDE_ID,OFFICE,YEAR,QUARTER,CATEGORY,YTD,AMOUNT,Unnamed: 7,PROGRAM,DESCRIPTION
359,A000014,HON. NEIL ABERCROMBIE,2009,2009Q3,FRANKED MAIL,"3,601.28","1,999.75",NaN,NaN,NaN
360,A000014,HON. NEIL ABERCROMBIE,2009,2009Q3,PERSONNEL COMPENSATION,"876,890.80","296,858.33",NaN,NaN,NaN
361,A000014,HON. NEIL ABERCROMBIE,2009,2009Q3,PERSONNEL BENEFITS,"1,490.13",0.00,NaN,NaN,NaN
362,A000014,HON. NEIL ABERCROMBIE,2009,2009Q3,TRAVEL,"51,897.02","22,006.86",NaN,NaN,NaN
363,A000014,HON. NEIL ABERCROMBIE,2009,2009Q3,"RENT, COMMUNICATION, UTILITIES","61,069.91","20,477.68",NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
4669,Z000017,HON. LEE M. ZELDIN,2018,Q1,OTHER SERVICES,4875.0,4875.0,NaN,NaN,NaN
4670,Z000017,HON. LEE M. ZELDIN,2018,Q1,SUPPLIES AND MATERIALS,3348.65,3348.65,NaN,NaN,NaN
4671,Z000017,HON. LEE M. ZELDIN,2018,Q1,EQUIPMENT,1156.32,1156.32,NaN,NaN,NaN
4672,Z000017,HON. LEE M. ZELDIN,2018,Q1,OFFICIAL EXPENSES OF MEMBERS TOTALS:,279487.46,279487.46,NaN,NaN,NaN


[election data](https://dataverse.harvard.edu/dataset.xhtml?persistentId=doi:10.7910/DVN/IG0UN2)

In [33]:
election_data = pd.read_csv('house-expenditure/1976-2020-house.csv')

# filter out the years before 2009 and after 2018
election_data = election_data[election_data['year'] >= 2009]
election_data = election_data[election_data['year'] <= 2018]

election_data


,year,state,state_po,state_fips,state_cen,state_ic,office,district,stage,runoff,special,candidate,party,writein,mode,candidatevotes,totalvotes,unofficial,version,fusion_ticket
22553,2010,ALABAMA,AL,1,63,41,US HOUSE,1,GEN,NaN,False,DAVID WALTER,CONSTITUTION,False,TOTAL,26357,156281,False,20220331,False
22554,2010,ALABAMA,AL,1,63,41,US HOUSE,1,GEN,NaN,False,JO BONNER,REPUBLICAN,False,TOTAL,129063,156281,False,20220331,False
22555,2010,ALABAMA,AL,1,63,41,US HOUSE,1,GEN,NaN,False,WRITEIN,NaN,True,TOTAL,861,156281,False,20220331,False
22556,2010,ALABAMA,AL,1,63,41,US HOUSE,2,GEN,NaN,False,BOBBY BRIGHT,DEMOCRAT,False,TOTAL,106865,219028,False,20220331,False
22557,2010,ALABAMA,AL,1,63,41,US HOUSE,2,GEN,NaN,False,MARTHA ROBY,REPUBLICAN,False,TOTAL,111645,219028,False,20220331,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29631,2018,WYOMING,WY,56,83,68,US HOUSE,0,GEN,NaN,False,DANIEL CLYDE CUMMINGS,CONSTITUTION,False,TOTAL,6070,201245,False,20220331,False
29632,2018,WYOMING,WY,56,83,68,US HOUSE,0,GEN,NaN,False,GREG HUNTER,DEMOCRAT,False,TOTAL,59903,201245,False,20220331,False
29633,2018,WYOMING,WY,56,83,68,US HOUSE,0,GEN,NaN,False,LIZ CHENEY,REPUBLICAN,False,TOTAL,127963,201245,False,20220331,False
29634,2018,WYOMING,WY,56,83,68,US HOUSE,0,GEN,NaN,False,RICHARD BRUBAKER,LIBERTARIAN,False,TOTAL,6918,201245,False,20220331,False
